## Backtesting Sesion 1
### Análisis de Propiedades de Señales
En este cuaderno analizamos más en detalle las señales del oscilador estocástico
para mostrar las principales características de una estrategia
de trading. La idea central es
- Con independencia de la señal utilizada, si utilizamos algoritmos con señales de entrada/salida, podemos
calcular las distribuciones de sus principales propiedades, por ejemplo:
  - duración media del trade
  - porcentaje de trades en ganancias/pérdidas
  - rentabilidad media de los trades positivos/negativos
- Se puede analizar la consistencia de estas distribuciones con los cambios de parámetros

____

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Datos 
Asumimos que tenemos un dataset disponible con datos de mercado.
Partimos de un snapshot de acciones del IBEX35

In [ ]:
import pickle
with open('../data/stock_data.pkl', 'rb') as handle:
    stock_data = pickle.load(handle)

Construimos un único dataframe para guardar los datos de cierre de todas las acciones

In [ ]:
close_dict = {tk: df.close for tk,df in stock_data.items()}
stock_close = pd.DataFrame(close_dict)
stock_close.head()

Seleccionamos, por simplicidad un conjunto de tickers para trabajar

In [ ]:
ticker_list = ['BBVA','SAN','REP','TEF','IBE','FER','ITX','ACS','AMS','GRF']
stock_close = stock_close[ticker_list]
stock_close

___

Las funciones desarrolladas anteriormente para el oscilador estocástico están accesibles
en una clase programada en el fichero stoosc.py

In [ ]:
from stoosc import Sto # clase con las funciones del oscilador estocastico

In [ ]:
ticker = 'FER' 

In [ ]:
sto_states = Sto.stochastic_osc_states(stock_close[ticker])
sto_states.value_counts()

Calculamos los eventos de compra y venta

In [ ]:
events = sto_states.diff()
events

___

Ajustamos los eventos inicial y final
- Marcamos con 1 (compra) si el estado del primer día es invertido
- Marcamos con -1 (venta) si el estado del último día es invertido

In [ ]:
events.iloc[0] = sto_states.iloc[0]
if sto_states.iloc[-1] == 1:
    events.iloc[-1] = -1
events.value_counts()

In [ ]:
only_events = events[events != 0]
only_events

____
Dado que las entradas y salidas están pareadas,
podemos calcular la **duración** del trade como el número de 
sesiones transcurridas entre ambos eventos


In [ ]:
delta = (only_events.index[1] - only_events.index[0])

In [ ]:
delta.days

Recorremos los trades y calculamos las propiedades

In [ ]:
prices = stock_close[ticker]
trades = []
for i in range(0, only_events.shape[0], 2):
    trade_ret = prices[only_events.index[i+1]] / prices[only_events.index[i]] - 1
    trade = {
        'day_in': only_events.index[i],
        'day_out': only_events.index[i+1],
        'ret': trade_ret,
        'duration': (only_events.index[i+1] - only_events.index[i]).days,
    }
    trades.append(pd.Series(trade))

In [ ]:
trades_df = pd.DataFrame(trades)
trades_df

____
Vemos la distribución de los rendimientos y la duración

In [ ]:
trades_df.describe()

In [ ]:
trades_df.plot.scatter('duration','ret')

In [ ]:
result = (trades_df.ret >= 0).astype(int)
result

In [ ]:
result.value_counts()/result.shape[0]

____
A partir del código anterior generamos una función que nos calcula el dataframe
con las fechas de entrada/salida, y el rendimiento

In [ ]:
def sto_trades_df(prices, win=20, obought=0.8, osold=0.2):
    sto_states = Sto.stochastic_osc_states(prices, win=win, obought=obought, osold=osold)
    
    events = sto_states.diff()
    events.iloc[0] = sto_states.iloc[0]
    if sto_states.iloc[-1] == 1:
        events.iloc[-1] = -1
    only_events = events[events != 0]
    
    trades = []
    for i in range(0, only_events.shape[0], 2):
        trade_ret = prices[only_events.index[i+1]] / prices[only_events.index[i]] - 1
        trade = {
            'day_in': only_events.index[i],
            'day_out': only_events.index[i+1],
            'ret': trade_ret,
            'duration': (only_events.index[i+1] - only_events.index[i]).days,
        }
        trades.append(pd.Series(trade))
    return pd.DataFrame(trades)  

In [ ]:
sto_trades_df(stock_close['TEF'])

____
Ahora, utilizamos la función para calcular las propiedades en 
las diferentes acciones

In [ ]:
list_rets = []
list_durations = []

for tk in stock_close.columns:
    print(tk)
    tk_df = sto_trades_df(stock_close[tk])
    list_durations.append(tk_df['duration'])
    list_rets.append(tk_df['ret'])

In [ ]:
all_rets = pd.concat(list_rets)
sns.distplot(all_rets)

In [ ]:
all_rets.describe()

Esta distribución muestra muchos valores extremos del lado negativo.
Aproximamos la función de acumulada para ver la proporción de observaciones
hasta un nivel de retorno

In [ ]:
cum_proportion = pd.DataFrame({
    'ret': all_rets.sort_values(),
    'cum_prob': np.arange(1, all_rets.shape[0] + 1)/all_rets.shape[0]
})
cum_proportion.head()

por ejemplo a 3 desviaciones estándar de la media obtenemos

In [ ]:
val = all_rets.mean() - 3*all_rets.std()
val

In [ ]:
cum_proportion[cum_proportion.ret <= val]

El equivalente en la distribución normal corresponde a:ºm

In [ ]:
from scipy.stats import norm
norm.cdf(-3)

___

Respecto a las duraciones, comparamos por separado las ganadoras de las perdedoras

In [ ]:
all_durations = pd.concat(list_durations)
duration_class = pd.concat([all_durations, all_rets >= 0], axis=1)
duration_class.head()

In [ ]:
sns.displot(duration_class, x='duration', hue='ret', stat='density')

____

### Análisis de Sensibilidad 
Podemos analizar como cambian las distribuciones de las propiedades
respecto al cambio de un parámetro de nuestra estrategia. Por ejemplo,
nos interesa saber:
- la media y desviación de la duración según el cambio de la ventana
- la media y desviación de los rendimientos según el cambio de la ventana

In [ ]:
ticker = 'FER'
dflist = []
for iwin in range(15, 91, 5):
    print('window:', iwin)
    df = sto_trades_df(stock_close[ticker], win=iwin, obought=0.8, osold=0.2)
    df['window'] = f'w{iwin}' 
    dflist.append(df)
all_df = pd.concat(dflist)
dur_wins = all_df[['ret','duration','window']]
dur_wins

**Distribuciones de la duración** según la ventana del oscilador

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(data=dur_wins, x='duration', y='window', ax=ax)

**Distribuciones de los rendimientos** según el tamaño de la ventana del oscilador estocástico 

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
sns.boxplot(data=dur_wins, x='ret', y='window', ax=ax)

____
### Ejercicios Propuesto 
 - Analizar la sensibilidad de la distribución de rendimientos respecto a cambios en los parámetros de sobrecompra (ejemplo de 0.95 a 0.65)
 - Repetir el análisis sobre la distribución de duración de trades considerando todas las empresas del ticker list.